In [1]:
pwd

'/content'

In [2]:
!pip install keras
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='AiZRS_D-r5y6Pgg_FB2No8ZsrHi3bu1VSOm0p0LK6DzV',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'cnnecgvj-donotdelete-pr-4ralnc6ghus6ny'
object_key = 'data.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [ ]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [6]:
pwd

'/content'

In [ ]:
import os
filenames = os.listdir('/home/wsuser/work/data/train')

In [9]:
x_train = train_datagen.flow_from_directory("/content/drive/MyDrive/Data set/Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation/data/test",target_size = (64,64),batch_size = 32,class_mode = "categorical")
x_test = test_datagen.flow_from_directory("/content/drive/MyDrive/Data set/Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation/data/train",target_size = (64,64),batch_size = 32,class_mode = "categorical")

Found 0 images belonging to 6 classes.
Found 662 images belonging to 6 classes.


In [10]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [11]:
x_train.class_indices

{'Left Bundle Branch Block': 0,
 'Normal': 1,
 'Premature Atrial Contraction': 2,
 'Premature Ventricular Contractions': 3,
 'Right Bundle Branch Block': 4,
 'Ventricular Fibrillation': 5}

In [12]:
#MODEL BUILDING
model = Sequential()

In [15]:
model.add(Conv2D(32,(3,3),input_shape = (64,64,3),activation = "relu"))

In [13]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [14]:
model.add(Conv2D(32,(3,3),activation='relu'))

In [16]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [17]:
model.add(Flatten()) # ANN Input...

In [18]:
model.add(Dense(32))
model.add(Dense(6, activation='softmax'))

In [ ]:
model.summary()

In [20]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))

In [ ]:
model.save('ECG.h5')

In [24]:
!tar -zcvf ECG-arrhythmia-classification-model_new.tgz ECG.h5


tar: ECG.h5: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


In [25]:
ls -1

drive/
ECG-arrhythmia-classification-model_new.tgz
sample_data/


In [26]:
!pip install watson-machine-learning-client --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 538 kB 13.6 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 132 kB 67.6 MB/s 
     |████████████████████████████████| 79 kB 6.3 MB/s 
     |████████████████████████████████| 9.9 MB 77.3 MB/s 
     |████████████████████████████████| 140 kB 71.7 MB/s 
     |████████████████████████████████| 956 kB 68.5 MB/s 
     |████████████████████████████████| 135 kB 50.8 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.12.0-py3-none-any.whl size=73931 sha256=636b4ed0208c086ebde1f61dda99a23a0aeaa33a91e5c7aa54411dc31bfb787e
  Stored in directory: /root/.cache/pip/wheels/ec/94/29/2b57327cf00664b6614304f7958abd29d77ea0e5bbece2ea57
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.12.0-py3-none-any.whl size=562962 sha256=d9cde6cfe0

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"0FMoFudKPCRM351dg6DBvpLNZae1cjKCCPF_lbmrVuAi"
}
client=APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space ['resources'] if item['entity'] ["name"] == space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name( client,'CNN_ECG' )
print("Space UID = " + space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model='ECG-arrhythmia-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN_ECG",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_id(model_details)

In [ ]:
model_id

In [ ]:
client.repository.download(model_id,'my_model.tar1.gz')


In [33]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('ECG.h5')

In [34]:
pwd

'/content'

In [35]:
ls

drive/  ECG-arrhythmia-classification-model_new.tgz  sample_data/


In [36]:
img=image.load_img(r'/content/drive/MyDrive/photo/fig_21.png',target_size=(64,64))

In [37]:
import numpy as ns
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)

In [38]:
pred = model.predict(x)

1/1 [==============================] - 1s 750ms/step


In [39]:
pred[0]

array([0., 0., 0., 0., 0., 1.], dtype=float32)

In [40]:
index=['left Bundle Branch block',
       'Normal',
       'Premature Atrial Contraction',
       'Premature Ventricular Contraction',
       'Right Bundle Branch Block',
       'Ventricular Fibrillation']

In [ ]:
result = str(index[y_pred])
result

right  bundle block